## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# mu 101.5399 700

In [75]:
symbol = 'COST'

# 5 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '5m', skip_dl=False) # skip_dl=True on redo's
print(f'{symbol} 5m Interval Timestamp: {time_stamp} ')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 15 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '15m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 15m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 30 Minutes Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '30m', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 30m Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Hour Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1h', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1h Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Day Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1d', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1d Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

# 1 Week Time Interval
time_stamp, summary_table = f.dl_tf_pd(symbol, '1wk', skip_dl=False) # skip_dl=True on redo's
print(f'\n{symbol} 1wk Interval Timestamp: {time_stamp}')
col_headers = summary_table.model.tolist()
col_headers
summary_table_transposed = summary_table.T.iloc[1:]
summary_table_transposed.columns = col_headers
display(summary_table_transposed)

COST 5m Interval Timestamp: 2024-10-12 03:02:16 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.475,0.949608,0.949565,0.829768
prob_up,0.0,0.001323,0.007039,0.016896
prob_static,0.375,0.034671,0.028986,0.104699
prob_down,0.625,0.964006,0.963975,0.878406
precision,0.810945,0.795337,0.783042,0.677741
recall,0.78744,0.741546,0.758454,0.492754
f1,0.79902,0.7675,0.770552,0.570629
support,"[414.0, 415.0, 414.0]","[414.0, 415.0, 414.0]","[414.0, 415.0, 414.0]","[414.0, 415.0, 414.0]"



COST 15m Interval Timestamp: 2024-10-12 03:02:22


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.65,0.73452,0.86213,0.454653
prob_up,0.0,0.003749,0.009868,0.049467
prob_static,0.25,0.18588,0.08861,0.340067
prob_down,0.75,0.810371,0.901522,0.610466
precision,0.724138,0.695035,0.678322,0.683168
recall,0.7,0.653333,0.646667,0.46
f1,0.711864,0.67354,0.662116,0.549801
support,"[149.0, 148.0, 150.0]","[149.0, 148.0, 150.0]","[149.0, 148.0, 150.0]","[149.0, 148.0, 150.0]"



COST 30m Interval Timestamp: 2024-10-12 03:02:26


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,down,down
kelly_1:2.5,0.3,0.81979,0.611061,0.539347
prob_up,0.125,0.002426,0.023313,0.0725
prob_static,0.5,0.126295,0.254501,0.256538
prob_down,0.375,0.871278,0.722187,0.670962
precision,0.715867,0.780198,0.758483,0.636605
recall,0.766798,0.780198,0.752475,0.475248
f1,0.740458,0.780198,0.755467,0.544218
support,"[503.0, 506.0, 505.0]","[503.0, 506.0, 505.0]","[503.0, 506.0, 505.0]","[503.0, 506.0, 505.0]"



COST 1d Interval Timestamp: 2024-10-12 03:02:37


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,up,up,down
kelly_1:2.5,0.3,0.300322,0.228286,0.15355
prob_up,0.25,0.50023,0.448776,0.324316
prob_static,0.25,0.110706,0.182316,0.280291
prob_down,0.5,0.389065,0.368909,0.395393
precision,0.829167,0.829493,0.836538,0.643216
recall,0.808943,0.728745,0.704453,0.520325
f1,0.81893,0.775862,0.764835,0.575281
support,"[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]","[247.0, 250.0, 246.0]"



COST 1wk Interval Timestamp: 2024-10-12 03:02:42


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,down
kelly_1:2.5,0.3,0.542922,0.383076,0.327455
prob_up,0.5,0.051583,0.095955,0.247912
prob_static,0.25,0.274901,0.344705,0.232478
prob_down,0.25,0.673516,0.55934,0.519611
precision,0.829268,0.678571,0.701754,0.648649
recall,0.68,0.791667,0.833333,0.5
f1,0.747253,0.730769,0.761905,0.564706
support,"[50.0, 48.0, 48.0]","[50.0, 48.0, 48.0]","[50.0, 48.0, 48.0]","[50.0, 48.0, 48.0]"


## Sanity Checks

In [77]:
df = f.load('COST','5m')
df.tail(3)

,open,high,low,close,adj_close,volume,dividends,stock_splits
Datetime,,,,,,,,
2024-10-11 19:45:00-04:00,888.98,888.98,888.490,888.900,888.900,0,0.0,0.0
2024-10-11 19:50:00-04:00,888.98,888.98,888.385,888.385,888.385,0,0.0,0.0
2024-10-11 19:55:00-04:00,888.50,888.77,888.100,888.100,888.100,0,0.0,0.0


### Check Summary Output

In [47]:
symbol='NVDA'
interval='5m'
period='1mo'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval, period)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

DL Time: 2024-10-10 15:51:25


,model,prediction,kelly_1:2.5,prob_up,prob_static,prob_down,precision,recall,f1,support
0,XGBoost,up,0.475000,0.625000,0.125000,0.250000,0.789091,0.791971,0.790528,"[274.0, 231.0, 285.0]"
1,GradientBoosting,up,0.375125,0.553661,0.071753,0.374586,0.757143,0.773723,0.765343,"[274.0, 231.0, 285.0]"
2,RandomForest,up,0.455799,0.611285,0.076480,0.312235,0.740741,0.729927,0.735294,"[274.0, 231.0, 285.0]"
3,KNN,up,0.347789,0.534135,0.148656,0.317209,0.658915,0.620438,0.639098,"[274.0, 231.0, 285.0]"


In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')